In [ ]:
#point to local path for lotus package directory
import sys
sys.path.append(r'C:\Users\Kingdel\Documents\GitHub')

from lotus import * #bulk import statement to import standard and custom libraries

In [ ]:
#dmm serial port setup
ser=serial.Serial('COM3',timeout=3.0)
ser.flushInput()

#experimentally determined gain factor
lpt_gain=0.000595 #inches per ohm

In [ ]:
num_pts=100
lcr_df=lcr_sweep(num_pts)   

plot_sweep(lcr_df)
lcr_df.head(5)

In [ ]:
rm,E4980AL=lcr_init()

freq=3000 #test frequency, in Hz

E4980AL.write(':FREQuency %f' % (freq))

In [ ]:
interrupt=False #variable to control data loop
data=pd.DataFrame([],columns=['posn(in)','output(uH)','res(ohm)']) #blank dataframe to hold data

#setup plot
p1=figure(plot_width=650,plot_height=350)
t=show(p1,notebook_handle=True)

button=widgets.Button(description='start data logging')
display(button)

def when_clicked(b): #ugly hack to get real-time datalogging
    def get_data():
        global interrupt
        global data
        while not interrupt:     
                
                #potentiometer resistance
                cmd='READ?'
                res=dmm_cmd(cmd,ser,0.1)
                res=float(res.strip()) 
                x=res*lpt_gain
                
                # sensor voltage
                Ls,Rs=lcr_data(E4980AL)
                y = Ls*1e6
                              
                data.loc[len(data)]=[x,y,Rs]           
                r1=p1.circle(x,y,size=7)
                push_notebook(handle=t)

#                 time.sleep(0.5)
    thread = threading.Thread(target=get_data)
    thread.start()

button.on_click(when_clicked)


In [ ]:
output_dir=r'C:\Users\Kingdel\Desktop\data'
os.chdir(output_dir)

filename='one_inch_3B.csv'

In [ ]:
button2=widgets.Button(description='stop data logging')
display(button2)
def stop_thread(c):
    global interrupt
    interrupt=True
    
    data.to_csv(filename)
    lcr_df.to_csv('sweep_'+filename)
    
    
button2.on_click(stop_thread)

In [ ]:
filename